# [CDAF] Atividade 5

## Nome e matrícula
Nome: Lucas Affonso Pires
Matrícula: 2023028420

## Introdução
- Neste notebook, vamos implementar o carregamento dos dados no formato SPADL
- Um modelo de Expected Threat
- Um modelo VAEP com pipeline completa

## Dados
https://figshare.com/collections/Soccer_match_event_dataset/4415000

### Carregando os dados

In [1]:
import numpy as np
import pandas as pd

In [2]:
def load_matches(path):
    matches = pd.read_json(path_or_buf=path)
    # as informações dos times de cada partida estão em um dicionário dentro da coluna 'teamsData', então vamos separar essas informações
    team_matches = []
    for i in range(len(matches)):
        match = pd.DataFrame(matches.loc[i, 'teamsData']).T
        match['matchId'] = matches.loc[i, 'wyId']
        team_matches.append(match)
    team_matches = pd.concat(team_matches).reset_index(drop=True)

    return team_matches

In [3]:
def load_players(path):
    players = pd.read_json(path_or_buf=path)
    players['player_name'] = players['firstName'] + ' ' + players['lastName']
    players = players[['wyId', 'player_name']].rename(columns={'wyId': 'player_id'})

    return players

In [4]:
def load_events(path):
    events = pd.read_json(path_or_buf=path)
    # pré processamento em colunas da tabela de eventos para facilitar a conversão p/ SPADL
    events = events.rename(columns={
        'id': 'event_id',
        'eventId': 'type_id',
        'subEventId': 'subtype_id',
        'teamId': 'team_id',
        'playerId': 'player_id',
        'matchId': 'game_id'
    })
    events['milliseconds'] = events['eventSec'] * 1000
    events['period_id'] = events['matchPeriod'].replace({'1H': 1, '2H': 2})

    return events

In [5]:
def load_minutes_played_per_game(path):
    minutes = pd.read_json(path_or_buf=path)
    minutes = minutes.rename(columns={
        'playerId': 'player_id',
        'matchId': 'game_id',
        'teamId': 'team_id',
        'minutesPlayed': 'minutes_played'
    })
    minutes = minutes.drop(['shortName', 'teamName', 'red_card'], axis=1)

    return minutes

In [6]:
leagues = ['England', 'Spain']
events = {}
matches = {}
minutes = {}
for league in leagues:
    path = r'matches_{}.json'.format(league)
    matches[league] = load_matches(path)
    path = r'events_{}.json'.format(league)
    events[league] = load_events(path)
    path = r'minutes_played_per_game_{}.json'.format(league)
    minutes[league] = load_minutes_played_per_game(path)

<ipython-input-4-92aadb3a4d0c>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events['period_id'] = events['matchPeriod'].replace({'1H': 1, '2H': 2})
<ipython-input-4-92aadb3a4d0c>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events['period_id'] = events['matchPeriod'].replace({'1H': 1, '2H': 2})


In [7]:
path = r'players.json'
players = load_players(path)
players['player_name'] = players['player_name'].str.decode('unicode-escape')

### SPADL

In [8]:
!pip install git+https://github.com/ML-KULeuven/socceraction.git
!pip install numpy==1.26.4 --force-reinstall

  Cloning https://github.com/ML-KULeuven/socceraction.git to /tmp/pip-req-build-xsexh1m4
  Running command git clone --filter=blob:none --quiet https://github.com/ML-KULeuven/socceraction.git /tmp/pip-req-build-xsexh1m4
  Resolved https://github.com/ML-KULeuven/socceraction.git to commit 910a1840b1ef933848e711c4ee2049dccd3ac764
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.

In [9]:
!pip install multimethod==1.9.1 --force-reinstall

  Using cached multimethod-1.9.1-py3-none-any.whl.metadata (9.2 kB)
Using cached multimethod-1.9.1-py3-none-any.whl (10 kB)
  Attempting uninstall: multimethod
    Found existing installation: multimethod 1.9.1
    Uninstalling multimethod-1.9.1:
      Successfully uninstalled multimethod-1.9.1


In [10]:
from tqdm import tqdm
import socceraction.spadl as spd

/usr/local/lib/python3.11/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [11]:
def spadl_transform(events, matches):
    spadl = []
    game_ids = events.game_id.unique().tolist()
    for g in tqdm(game_ids):
        match_events = events.loc[events.game_id == g]
        match_home_id = matches.loc[(matches.matchId == g) & (matches.side == 'home'), 'teamId'].values[0]
        match_actions = spd.wyscout.convert_to_actions(events=match_events, home_team_id=match_home_id)
        match_actions = spd.play_left_to_right(actions=match_actions, home_team_id=match_home_id)
        match_actions = spd.add_names(match_actions)
        spadl.append(match_actions)
    spadl = pd.concat(spadl).reset_index(drop=True)

    return spadl

In [23]:
spadl = {}
for league in leagues:
    spadl[league] = spadl_transform(events=events[league], matches=matches[league])

100%|██████████| 380/380 [03:47<00:00,  1.67it/s]


## Parte I
- Vamos implementar um modelo de xT usando a biblioteca Socceraction, referenciada abaixo

## Referências
- [1] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.xthreat.ExpectedThreat.html#socceraction.xthreat.ExpectedThreat
- [2] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.xthreat.get_successful_move_actions.html#socceraction.xthreat.get_successful_move_actions
- [3] https://socceraction.readthedocs.io/en/latest/documentation/valuing_actions/xT.html

### Questão 1

- Instancie um objeto ExpectedThreat [2] com parâmetros l=25 e w=16.
- Faça o fit do modelo ExpectedThreat com o dataframe "spadl".

In [25]:
from socceraction import xthreat as xt

In [26]:
from socceraction.xthreat import ExpectedThreat

# Instanciar o modelo xT
xt_model = ExpectedThreat(l=25, w=16)

# Concatenar todos os dataframes SPADL (Inglaterra + Espanha) para treinar o modelo
all_spadl = pd.concat(list(spadl.values())).reset_index(drop=True)

# Treinar o modelo
xt_model.fit(all_spadl)


# iterations:  29


### Questão 2
- Crie um dataframe "prog_actions" à partir do dataframe "spadl", contendo apenas as ações de progressão e que são bem-sucedidas [3].
- Use o método rate do objeto ExpectedThreat p/ calcular o valor de cada ação de progressão do dataframe "prog_actions", em uma coluna chamada "action_value".
- Agrupe o dataframe "prog_actions" por "player_name" e reporte a soma dos "action_value".
- Reporte os 10 jogadores com maior "action_value".

In [49]:
from socceraction.spadl import config

prog_actions = spadl['Spain']
prog_actions = prog_actions[(prog_actions['type_name'].isin(['pass', 'cross', 'dribble'])) &
    (prog_actions['result_name'] == 'success') &
    (prog_actions['end_x'] > prog_actions['start_x'])  # só ações que avançam no campo
]

prog_actions['action_value'] = xt_model.rate(prog_actions)

prog_actions = prog_actions.merge(players, how='left', on='player_id')
melhores = prog_actions.groupby('player_name')['action_value'].sum().sort_values(ascending=False).reset_index()

print(melhores.head(10))

                      player_name  action_value
0  Lionel Andrés Messi Cuccittini     12.673312
1  Marcelo Vieira da Silva Júnior     10.840617
2              Hugo Mallo Novegil      8.145843
3   Éver Maximiliano David Banega      8.103392
4   Juan Francisco Moreno Fuertes      7.705466
5                    Ivan Rakitić      7.349526
6     Asier Illarramendi Andonegi      7.276947
7     Roberto José Rosales Altuve      7.117482
8       Álvaro Odriozola Arzallus      7.032706
9       José Luis Morales Nogales      6.989071


<ipython-input-49-fcdc3662197d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prog_actions['action_value'] = xt_model.rate(prog_actions)


## Parte II
- Nessa atividade, temos implementada a pipeline inteira do VAEP [1] para os dados do Wyscout das Top 5 ligas.
- [2] é a documentação das funções do VAEP na API do socceraction.
- [3] apresenta uma explicação do framework com uma mistura de intuição, matemática e código.
- [4] são notebooks públicos que implementam o VAEP para outro conjunto de dados.

## Referências
- [1] https://tomdecroos.github.io/reports/kdd19_tomd.pdf
- [2] https://socceraction.readthedocs.io/en/latest/api/vaep.html
- [3] https://socceraction.readthedocs.io/en/latest/documentation/valuing_actions/vaep.html
- [4] https://github.com/ML-KULeuven/socceraction/tree/master/public-notebooks

## Instruções
- Para cada header do notebook abaixo, vocês devem explicar o que foi feito e à qual seção/subseção/equação do paper "Actions Speak Louder than Goals: Valuing Actions by Estimating Probabilities" ela corresponde. Justifique suas respostas.
- Além disso, após algumas partes do código haverão perguntas que vocês devem responder, possivelmente explorando minimamente o que já está pronto.
- Por fim, vocês devem montar um diagrama do fluxo de funções/tarefas de toda a pipeline do VAEP abaixo. Esse diagrama deve ser enviado como arquivo na submissão do Moodle, para além deste notebook.

### Features

In [27]:
from socceraction.vaep import features as ft

In [28]:
def features_transform(spadl):
    spadl.loc[spadl.result_id.isin([2, 3]), ['result_id']] = 0
    spadl.loc[spadl.result_name.isin(['offside', 'owngoal']), ['result_name']] = 'fail'

    xfns = [
        ft.actiontype_onehot,
        ft.bodypart_onehot,
        ft.result_onehot,
        ft.goalscore,
        ft.startlocation,
        ft.endlocation,
        ft.team,
        ft.time,
        ft.time_delta
    ]

    features = []
    for game in tqdm(np.unique(spadl.game_id).tolist()):
        match_actions = spadl.loc[spadl.game_id == game].reset_index(drop=True)
        match_states = ft.gamestates(actions=match_actions)
        match_feats = pd.concat([fn(match_states) for fn in xfns], axis=1)
        features.append(match_feats)
    features = pd.concat(features).reset_index(drop=True)

    return features

1- O que a primeira e a segunda linhas da função acima fazem? Qual sua hipótese sobre intuito dessas transformações? Como você acha que isso pode impactar o modelo final?

Elas servem para retirar as instâncias de gol contra e impedimento, garantindo que somente gols válidos sejam analisados.

In [29]:
features = {}
for league in ['England', 'Spain']:
    features[league] = features_transform(spadl[league])

  0%|          | 0/380 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/socceraction/vaep/features.py:94: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.shift(i, fill_value=float("nan")).fillna(x.iloc[0])  # noqa: B023
/usr/local/lib/python3.11/dist-packages/socceraction/vaep/features.py:94: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  lambda x: x.shift(i, fill_value=float("nan")).fillna(x.iloc[0])  # noqa: B023
/usr/local/lib/python3.11/dist-packages/socceraction/vaep/features.py:94: FutureWarning: Downcasting object dtype arrays on .

### Labels

In [30]:
import socceraction.vaep.labels as lab

In [31]:
def labels_transform(spadl):
    yfns = [lab.scores, lab.concedes]

    labels = []
    for game in tqdm(np.unique(spadl.game_id).tolist()):
        match_actions = spadl.loc[spadl.game_id == game].reset_index(drop=True)
        labels.append(pd.concat([fn(actions=match_actions) for fn in yfns], axis=1))

    labels = pd.concat(labels).reset_index(drop=True)

    return labels

In [32]:
labels = {}
for league in ['England', 'Spain']:
    labels[league] = labels_transform(spadl[league])

100%|██████████| 380/380 [00:27<00:00, 13.90it/s]


In [33]:
labels['England']['scores'].sum()

7400

In [34]:
labels['England']['concedes'].sum()

2465

2- Explique o por que da quantidade de labels positivos do tipo scores ser muito maior que do concedes. Como você acha que isso pode impactar o modelo final?

Porque todas as ações ofensivas (passes, finalizações, dribles) aumentam a chance de marcar. Já o concedes só se torna "positivo" em transições defensivas mal sucedidas, que são menos frequentes e mais difíceis de capturar em uma sequência curta (10 ações). Ou seja, a maioria das jogadas ofensivas gera potencial de ataque, enquanto apenas algumas situações levam diretamente a um gol adversário.

### Training Model

In [35]:
import xgboost as xgb
import sklearn.metrics as mt

In [36]:
def train_vaep(X_train, y_train, X_test, y_test):
    models = {}
    for m in ['scores', 'concedes']:
        models[m] = xgb.XGBClassifier(random_state=0, n_estimators=50, max_depth=3)

        print('training ' + m + ' model')
        models[m].fit(X_train, y_train[m])

        p = sum(y_train[m]) / len(y_train[m])
        base = [p] * len(y_train[m])
        y_train_pred = models[m].predict_proba(X_train)[:, 1]
        train_brier = mt.brier_score_loss(y_train[m], y_train_pred) / mt.brier_score_loss(y_train[m], base)
        print(m + ' Train NBS: ' + str(train_brier))
        print()

        p = sum(y_test[m]) / len(y_test[m])
        base = [p] * len(y_test[m])
        y_test_pred = models[m].predict_proba(X_test)[:, 1]
        test_brier = mt.brier_score_loss(y_test[m], y_test_pred) / mt.brier_score_loss(y_test[m], base)
        print(m + ' Test NBS: ' + str(test_brier))
        print()

        print('----------------------------------------')

    return models

In [37]:
models = train_vaep(X_train=features['England'], y_train=labels['England'], X_test=features['Spain'], y_test=labels['Spain'])

training scores model
scores Train NBS: 0.8455802930197476

scores Test NBS: 0.8508400022275144

----------------------------------------
training concedes model
concedes Train NBS: 0.9661436865979788

concedes Test NBS: 0.9766386802230537

----------------------------------------


3- Por que treinamos dois modelos diferentes? Por que a performance dos dois é diferente?

Conforme explicado anteriormente, o modelo concedes tem valores bem menores, já que apenas erros graves como perda de posse em posições perigosas, são considerados. Por isso, ao treinar os modelos, temos diferentes resultados para cada, logo, é melhor treina-los separadamente.

### Predictions

In [38]:
def generate_predictions(features, models):
    preds = {}
    for m in ['scores', 'concedes']:
        preds[m] = models[m].predict_proba(features)[:, 1]
    preds = pd.DataFrame(preds)

    return preds

In [39]:
preds = {}
preds['Spain'] = generate_predictions(features=features['Spain'], models=models)

### Action Values

In [40]:
import socceraction.vaep.formula as fm

In [62]:
def calculate_action_values(spadl, predictions):
    action_values = fm.value(actions=spadl, Pscores=predictions['scores'], Pconcedes=predictions['concedes'])
    action_values = pd.concat([
        spadl[['player_id', 'original_event_id', 'action_id', 'game_id', 'start_x', 'start_y', 'end_x', 'end_y', 'type_name', 'result_name']],
        predictions.rename(columns={'scores': 'Pscores', 'concedes': 'Pconcedes'}),
        action_values
    ], axis=1)

    return action_values

In [63]:
action_values = {}
action_values['Spain'] = calculate_action_values(spadl=spadl['Spain'], predictions=preds['Spain'])

4- Explore as ações com Pscores >= 0.95. Por que elas tem um valor tão alto? As compare com ações do mesmo tipo e resultado opostado. Será que o modelo aprende que essa combinação de tipo de ação e resultado está diretamente relacionado à variável y que estamos tentando prever?

5- Qual formula do paper corresponde à coluna 'offensive_value' do dataframe action_values? E a coluna 'defensive_value'?

4- Ações com Pscores ≥ 0.95 são aquelas em que o modelo está quase certo de que um gol será marcado nos próximos 10 passos. Essas ações geralmente ocorrem: Dentro da grande área, com finalizações (shots) e com resultado = success (ex: gol ou finalização no alvo). Ações de chute bem-sucedidas em posições próximas ao gol → fortemente associadas a gols marcados nos dados → o modelo aprende isso e retorna Pscore alto.

5- offensive_value	= Eq. (2). Quanto a ação aumentou a chance de marcar gol.
defensive_value	= Eq. (3).	Quanto a ação reduziu a chance de sofrer gol.

### Player Ratings

In [64]:
def calculate_minutes_per_season(minutes_per_game):
    minutes_per_season = minutes_per_game.groupby('player_id', as_index=False)['minutes_played'].sum()

    return minutes_per_season

In [65]:
minutes_per_season = {}
minutes_per_season['Spain'] = calculate_minutes_per_season(minutes['Spain'])

In [66]:
def calculate_player_ratings(action_values, minutes_per_season, players):
    player_ratings = action_values.groupby(by='player_id', as_index=False).agg({'vaep_value': 'sum'}).rename(columns={'vaep_value': 'vaep_total'})
    player_ratings = player_ratings.merge(minutes_per_season, on=['player_id'], how='left')
    player_ratings['vaep_p90'] = player_ratings['vaep_total'] / player_ratings['minutes_played'] * 90
    player_ratings = player_ratings[player_ratings['minutes_played'] >= 600].sort_values(by='vaep_p90', ascending=False).reset_index(drop=True)
    player_ratings = player_ratings.merge(players, on=['player_id'], how='left')
    player_ratings = player_ratings[['player_id', 'player_name', 'minutes_played', 'vaep_total', 'vaep_p90']]

    return player_ratings

In [72]:
player_ratings = {}
player_ratings['Spain'] = calculate_player_ratings(action_values=action_values['Spain'], minutes_per_season=minutes_per_season['Spain'], players=players)
player_ratings['Spain'].head(10)

,player_id,player_name,minutes_played,vaep_total,vaep_p90
0,3359,Lionel Andrés Messi Cuccittini,3108.0,34.983645,1.013040
1,8278,Gareth Frank Bale,1850.0,14.496849,0.705252
2,3322,Cristiano Ronaldo dos Santos Aveiro,2355.0,17.426092,0.665965
3,3802,Philippe Coutinho Correia,1329.0,9.136238,0.618707
4,225946,Arnaldo Antonio Sanabria Ayala,902.0,6.177164,0.616347
5,3682,Antoine Griezmann,2591.0,17.362766,0.603106
6,3840,Iago Aspas Juncal,3038.0,19.836151,0.587641
7,280383,Enis Bardhi,1637.0,9.924908,0.545658
8,3425,Iker Muniain Goñi,816.0,4.932149,0.543987
9,22578,Nicola Sansone,672.0,4.035320,0.540445


6- Acha que o Top 5 da lista é bem representativo? Compare esse ranqueamento do VAEP com o do xT da Parte I. Qual você acha que é mais representativo?

Aparentemente sim. Os jogadores no top 5 são todos jogadores famosos considerados uns dos melhores do momento analisado. Esse ranqueamento parece considerar as ações ofensivas como mais valiosas, visto que quase todos os atletas do top 10 são atacantes ou meias ofensivos. O modelo xT parece ser mais inclusivo com jogadores de diversas posições, mostrando a participação em jogadas perigosas não somente com ações ofensivas.